In [1]:
import pandas as pd

In [15]:
df = pd.read_csv("Grunfeld.fil.csv", index_col=0, index_row)
df.columns

Index(['year', 'invest', 'value', 'capital'], dtype='object')

In [16]:
df.set_index(df.columns[0], inplace=True)

In [17]:
df.head()

,invest,value,capital
year,,,
1935,317.6,3078.5,2.8
1936,391.8,4661.7,52.6
1937,410.6,5387.1,156.9
1938,257.7,2792.2,209.2
1939,330.8,4313.2,203.4


In [20]:
df1 = df.iloc[:20]
df2 = df.iloc[20:40]
df3 = df.iloc[40:60]
df4 = df.iloc[60:80]

In [19]:
df1

,invest,value,capital
year,,,
1935,317.6,3078.5,2.8
1936,391.8,4661.7,52.6
1937,410.6,5387.1,156.9
1938,257.7,2792.2,209.2
1939,330.8,4313.2,203.4
1940,461.2,4643.9,207.2
1941,512.0,4551.2,255.2
1942,448.0,3244.1,303.7
1943,499.6,4053.7,264.1


In [26]:
df1.columns

Index(['invest', 'value', 'capital'], dtype='object')

In [21]:
df2

,invest,value,capital
year,,,
1935,209.9,1362.4,53.8
1936,355.3,1807.1,50.5
1937,469.9,2676.3,118.1
1938,262.3,1801.9,260.2
1939,230.4,1957.3,312.7
1940,361.6,2202.9,254.2
1941,472.8,2380.5,261.4
1942,445.6,2168.6,298.7
1943,361.6,1985.1,301.8


In [23]:
import statsmodels.api as sm

In [29]:
dfi=[df1,df2,df3,df4]
for i in dfi:
    y = i['invest']
    X = i[['value', 'capital']]
    X = sm.add_constant(X)
    model = sm.OLS(y,X).fit()
    print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 invest   R-squared:                       0.921
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     99.58
Date:                Fri, 02 Feb 2024   Prob (F-statistic):           4.10e-10
Time:                        14:50:56   Log-Likelihood:                -117.14
No. Observations:                  20   AIC:                             240.3
Df Residuals:                      17   BIC:                             243.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -149.7825    105.842     -1.415      0.1

In [32]:
from scipy.linalg import toeplitz
import numpy as np

In [41]:
for i in dfi:
    y = i['invest']
    X = i[['value', 'capital']]
    X = sm.add_constant(X)
    ols_resid = sm.OLS(y,X).fit().resid
    resid_fit = sm.OLS(np.asarray(ols_resid)[1:],sm.add_constant(np.asarray(ols_resid)[:-1])).fit()
    rho = resid_fit.params[1]
    order = toeplitz(range(len(ols_resid)))
    sigma = rho**order
    gls_model = sm.GLS(y,X, sigma=sigma)
    gls_results = gls_model.fit()
    print(gls_results.summary())

                            GLS Regression Results                            
Dep. Variable:                 invest   R-squared:                       0.884
Model:                            GLS   Adj. R-squared:                  0.871
Method:                 Least Squares   F-statistic:                     65.05
Date:                Fri, 02 Feb 2024   Prob (F-statistic):           1.08e-08
Time:                        15:09:00   Log-Likelihood:                -113.43
No. Observations:                  20   AIC:                             232.9
Df Residuals:                      17   BIC:                             235.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -41.0390     91.310     -0.449      0.6

In [43]:
gls_estimates = []
for i in dfi:
    y = i['invest']
    X = i[['value', 'capital']]
    X = sm.add_constant(X)
    ols_resid = sm.OLS(y,X).fit().resid
    resid_fit = sm.OLS(np.asarray(ols_resid)[1:],sm.add_constant(np.asarray(ols_resid)[:-1])).fit()
    rho = resid_fit.params[1]
    order = toeplitz(range(len(ols_resid)))
    sigma = rho**order
    gls_model = sm.GLS(y,X, sigma=sigma)
    gls_results = gls_model.fit()
    gls_estimates.append(gls_results.params)
    df_combined = pd.concat(gls_estimates, axis=1)

df_combined.columns = ['firm 1', 'firm 2', 'firm 3', 'firm 4']
print(df_combined)

            firm 1     firm 2     firm 3    firm 4
const   -41.039005 -75.463468 -18.264837 -6.403126
value     0.090701   0.216342   0.033245  0.078177
capital   0.408967   0.188059   0.139050  0.316142


In [45]:
resid = []
for i in dfi:
    y = i['invest']
    X = i[['value', 'capital']]
    X = sm.add_constant(X)
    ols_resid = sm.OLS(y,X).fit().resid
    resid.append(ols_resid)
    df_resid = pd.concat(resid, axis=1)

print(df_resid)

               0           1          2          3
year                                              
1935   99.136365   -0.088247  -2.860176  10.621704
1936  -34.007001   68.839101 -14.402421  10.424950
1937 -140.475010    5.114461  -5.174519  -7.403540
1938   -3.279741 -104.959552 -23.294737   7.302408
1939 -109.451507 -184.488376 -28.030849 -14.681853
1940  -19.909169  -73.438963  -0.562215  -2.315154
1941   24.118813    3.901187  40.749594   0.630735
1942   98.015716   -0.780466  16.035522  -1.581078
1943   67.755169  -53.902277 -23.719213 -13.459228
1944  100.032630  -88.522056 -26.780100  -7.779748
1945   35.122997  -98.925714   1.768123  11.756712
1946  103.903920   56.282024  58.737236 -16.132704
1947   15.183377   52.357347  43.935869  -6.823348
1948  -51.861563  139.836316  31.227129   6.615260
1949 -115.391331   26.010077 -23.552006  -7.379499
1950  -63.506485   35.280974 -37.510995   1.267939
1951 -119.395704  103.768984  -4.983022  39.501522
1952  -77.815516  144.035315   

In [49]:
correlation_matrix = df_resid.corr()

In [55]:
Re = correlation_matrix.values
ReT = np.transpose(Re)

array([[ 1.        , -0.26146134,  0.27929288, -0.27295213],
       [-0.26146134,  1.        ,  0.42810705,  0.33795352],
       [ 0.27929288,  0.42810705,  1.        , -0.06792257],
       [-0.27295213,  0.33795352, -0.06792257,  1.        ]])

In [69]:
LM = 10*((np.trace(np.dot(ReT,Re))-4))
LM

10.459422424818392

In [71]:
from scipy.stats import chi2
critical_LM = chi2.ppf(1-0.05, 6)
critical_LM

12.591587243743977

In [72]:
E = df_resid.values

In [74]:
ET = np.transpose(E)

In [80]:
W = 0.05*np.dot(ET,E)
W

array([[ 7160.29387056, -1967.0463656 ,   607.53313552,  -282.7564235 ],
       [-1967.0463656 ,  7904.6634394 ,   978.45025028,   367.84024052],
       [  607.53313552,   978.45025028,   660.82938851,   -21.37565073],
       [ -282.7564235 ,   367.84024052,   -21.37565073,   149.87221809]])

In [82]:
I20 = np.eye(20,20)
I20

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [84]:
omega_cap = np.kron(W, I20)
omega_cap

array([[7160.29387056,    0.        ,    0.        , ...,   -0.        ,
          -0.        ,   -0.        ],
       [   0.        , 7160.29387056,    0.        , ...,   -0.        ,
          -0.        ,   -0.        ],
       [   0.        ,    0.        , 7160.29387056, ...,   -0.        ,
          -0.        ,   -0.        ],
       ...,
       [  -0.        ,   -0.        ,   -0.        , ...,  149.87221809,
           0.        ,    0.        ],
       [  -0.        ,   -0.        ,   -0.        , ...,    0.        ,
         149.87221809,    0.        ],
       [  -0.        ,   -0.        ,   -0.        , ...,    0.        ,
           0.        ,  149.87221809]])

In [90]:
Xi = []
for i in dfi:
    x1 = i[['value', 'capital']]
    Xi.append(x1.values)
print(Xi)

[array([[3.0785e+03, 2.8000e+00],
       [4.6617e+03, 5.2600e+01],
       [5.3871e+03, 1.5690e+02],
       [2.7922e+03, 2.0920e+02],
       [4.3132e+03, 2.0340e+02],
       [4.6439e+03, 2.0720e+02],
       [4.5512e+03, 2.5520e+02],
       [3.2441e+03, 3.0370e+02],
       [4.0537e+03, 2.6410e+02],
       [4.3793e+03, 2.0160e+02],
       [4.8409e+03, 2.6500e+02],
       [4.9009e+03, 4.0220e+02],
       [3.5265e+03, 7.6150e+02],
       [3.2547e+03, 9.2240e+02],
       [3.7002e+03, 1.0201e+03],
       [3.7556e+03, 1.0990e+03],
       [4.8330e+03, 1.2077e+03],
       [4.9249e+03, 1.4305e+03],
       [6.2417e+03, 1.7773e+03],
       [5.5936e+03, 2.2263e+03]]), array([[1362.4,   53.8],
       [1807.1,   50.5],
       [2676.3,  118.1],
       [1801.9,  260.2],
       [1957.3,  312.7],
       [2202.9,  254.2],
       [2380.5,  261.4],
       [2168.6,  298.7],
       [1985.1,  301.8],
       [1813.9,  279.1],
       [1850.2,  213.8],
       [2067.7,  132.6],
       [1796.7,  264.8],
       [1625

In [93]:
X1 = Xi[0]
X2 = Xi[1]
X3 = Xi[2]
X4 = Xi[3]

In [138]:
X1 = sm.add_constant(X1)
X2 = sm.add_constant(X2)
X3 = sm.add_constant(X3)
X4 = sm.add_constant(X4)

In [139]:
from scipy.linalg import block_diag
X_stacked = block_diag(X1, X2, X3, X4)
X_stacked

array([[   1. , 3078.5,    2.8,    0. ,    0. ,    0. ,    0. ,    0. ,
           0. ,    0. ,    0. ,    0. ],
       [   1. , 4661.7,   52.6,    0. ,    0. ,    0. ,    0. ,    0. ,
           0. ,    0. ,    0. ,    0. ],
       [   1. , 5387.1,  156.9,    0. ,    0. ,    0. ,    0. ,    0. ,
           0. ,    0. ,    0. ,    0. ],
       [   1. , 2792.2,  209.2,    0. ,    0. ,    0. ,    0. ,    0. ,
           0. ,    0. ,    0. ,    0. ],
       [   1. , 4313.2,  203.4,    0. ,    0. ,    0. ,    0. ,    0. ,
           0. ,    0. ,    0. ,    0. ],
       [   1. , 4643.9,  207.2,    0. ,    0. ,    0. ,    0. ,    0. ,
           0. ,    0. ,    0. ,    0. ],
       [   1. , 4551.2,  255.2,    0. ,    0. ,    0. ,    0. ,    0. ,
           0. ,    0. ,    0. ,    0. ],
       [   1. , 3244.1,  303.7,    0. ,    0. ,    0. ,    0. ,    0. ,
           0. ,    0. ,    0. ,    0. ],
       [   1. , 4053.7,  264.1,    0. ,    0. ,    0. ,    0. ,    0. ,
           0. ,    0. , 

In [95]:
yi = []
for i in dfi:
    y1 = i['invest']
    yi.append(y1.values)
print(yi)

[array([ 317.6,  391.8,  410.6,  257.7,  330.8,  461.2,  512. ,  448. ,
        499.6,  547.5,  561.2,  688.1,  568.9,  529.2,  555.1,  642.9,
        755.9,  891.2, 1304.4, 1486.7]), array([209.9, 355.3, 469.9, 262.3, 230.4, 361.6, 472.8, 445.6, 361.6,
       288.2, 258.7, 420.3, 420.5, 494.5, 405.1, 418.8, 588.2, 645.5,
       641. , 459.3]), array([ 33.1,  45. ,  77.2,  44.6,  48.1,  74.4, 113. ,  91.9,  61.3,
        56.8,  93.6, 159.9, 147.2, 146.3,  98.3,  93.5, 135.2, 157.3,
       179.5, 189.6]), array([ 40.29,  72.76,  66.26,  51.6 ,  52.41,  69.41,  68.35,  46.8 ,
        47.4 ,  59.57,  88.78,  74.12,  62.68,  89.36,  78.98, 100.66,
       160.62, 145.  , 174.93, 172.49])]


In [105]:
y_stacked = np.hstack((yi[0], yi[1], yi[2], yi[3]))
y_stacked

array([ 317.6 ,  391.8 ,  410.6 ,  257.7 ,  330.8 ,  461.2 ,  512.  ,
        448.  ,  499.6 ,  547.5 ,  561.2 ,  688.1 ,  568.9 ,  529.2 ,
        555.1 ,  642.9 ,  755.9 ,  891.2 , 1304.4 , 1486.7 ,  209.9 ,
        355.3 ,  469.9 ,  262.3 ,  230.4 ,  361.6 ,  472.8 ,  445.6 ,
        361.6 ,  288.2 ,  258.7 ,  420.3 ,  420.5 ,  494.5 ,  405.1 ,
        418.8 ,  588.2 ,  645.5 ,  641.  ,  459.3 ,   33.1 ,   45.  ,
         77.2 ,   44.6 ,   48.1 ,   74.4 ,  113.  ,   91.9 ,   61.3 ,
         56.8 ,   93.6 ,  159.9 ,  147.2 ,  146.3 ,   98.3 ,   93.5 ,
        135.2 ,  157.3 ,  179.5 ,  189.6 ,   40.29,   72.76,   66.26,
         51.6 ,   52.41,   69.41,   68.35,   46.8 ,   47.4 ,   59.57,
         88.78,   74.12,   62.68,   89.36,   78.98,  100.66,  160.62,
        145.  ,  174.93,  172.49])

In [134]:
X_stacked_T = np.transpose(X_stacked)
omega_cap_inverse = np.linalg.inv(omega_cap)
XT_omegainv_X = np.dot(X_stacked_T, np.dot(omega_cap_inverse, X_stacked))
XT_omegainv_y = np.dot(X_stacked_T, np.dot(omega_cap_inverse, y_stacked))
beta_fgls = np.dot(np.linalg.inv(XT_omegainv_X), XT_omegainv_y)
np.set_printoptions(suppress=True, precision=4)
beta_fgls

array([-160.6864,    0.1205,    0.38  ,   21.164 ,    0.1304,    0.4485,
        -19.712 ,    0.0346,    0.1368,    0.9366,    0.0679,    0.3146])

In [137]:
est_asy_cov = np.linalg.inv(XT_omegainv_X)

In [142]:
standard_errors_fgls = np.sqrt(np.diag(est_asy_cov))
standard_errors_fgls

array([ 90.4098,   0.0219,   0.0331, 116.1752,   0.0574,   0.1225,
        26.5828,   0.0128,   0.0225,  11.5904,   0.0171,   0.0261])

In [143]:
# AI help used for above task [ols + fgls] "https://sl.bing.net/jkOK96IIami"